## Implementation of CNN from SCRATCH

In [102]:
import numpy as np
import math

In [113]:
class Conv2d:
    id_ = 0
    def __init__(self,input_x:np.ndarray,filter_shape:tuple,number_of_filters:int,padding:int=1,
                 stride:int=1,max_pool_shape:tuple=None,max_pool_stride:int=2):
        self.input_x = input_x # (channel,H,W)
        self.filter_shape = filter_shape # (F_h,F_W)
        self.number_of_filters = number_of_filters
        self.padding = padding
        self.stride = stride
        self.filters = None
        self.max_pool_shape = max_pool_shape
        self.max_pool_stride = max_pool_stride        
        self.id_ = Conv2d.id_ 
        Conv2d.id_ += 1
        
    def xavier_weight_init(self,shape:tuple,fan_in:int,fan_out:int) -> np.ndarray:
        return np.random.randn(*shape)*np.sqrt(2/(fan_in+fan_out))
        
    # forward
    def __call__(self) -> np.ndarray:
        # Padding
        input_x = None
        if self.padding != 0:
            ch0 = self.input_x[0]
            ch1 = np.pad(self.input_x[1],1)
            ch2 = np.pad(self.input_x[2],1)
            input_x = np.vstack([[ch0],[ch1],[ch2]])
        else:
            input_x = self.input_x
  
        
        # Calculating output Dimensions
        output_h = math.floor((((self.input_x.shape[1]+2*self.padding-self.filter_shape[1]) /self.stride) + 1 ))
        output_w = math.floor((((self.input_x.shape[2]+2*self.padding-self.filter_shape[2]) /self.stride) + 1 ))
        
        # Initializing output
        output = np.zeros((self.number_of_filters,output_h,output_w)) # (number_of_filters,H,W)
        
        # Initializing filters
        self.filters = [self.xavier_weight_init(
            shape=(input_x.shape[0],self.filter_shape[1],self.filter_shape[2]),
            fan_in = input_x.shape[0] * input_x.shape[1] * input_x.shape[2],
            fan_out = self.filter_shape[0] * self.filter_shape[1] * input_x.shape[0] if self.max_pool_shape is None else self.filter_shape[0] * self.filter_shape[1] * input_x.shape[0] /(self.max_pool_shape[0]*self.max_pool_shape[1])  
        ) for _ in range(self.number_of_filters)]
        
        
        # Convolving
        for index, filter_ in enumerate(self.filters):
            out = np.empty((output_h,output_w))
            row_counter = 0
            col_counter = 0
            for row in np.arange(0,input_x.shape[1],step=self.stride):
                for col in np.arange(0,input_x.shape[2],step=self.stride):
                    out[row_counter,col_counter] =(input_x[:,row:row+self.filter_shape[1],col:colself.filter_shape[2]] * filter_).sum()
                    col_counter += 1
                row_counter += 1
            output[index] = out
        
        
        # Max Pooling       
        if self.max_pool_shape is not None:
            pooling_output = np.zeros((output.shape[0],output.shape[1]//self.max_pool_shape[0],output.shape[2]//self.max_pool_shape[1]))
            for i in range(output.shape[0]):
                col_counter = 0
                row_counter = 0
                for row in np.arange(0,input_x.shape[1],step=self.max_pool_stride):
                    for col in np.arange(0,input_x.shape[2],step=self.max_pool_stride):
                        pooling_output[i,row_counter,col_counter] = output[i,row:row+self.max_pool_shape[0],col:col+self.max_pool_shape[1]].max()
                        col_counter += 1 
                    row_counter += 1
                        
            return pooling_output
        
        return output
        